# RAG with Galileo, LangChain and GPT
Retrieval-Augmented Generation (RAG) is an architectural approach that can enhance the effectiveness of large language model (LLM) applications using customized data. In this example, we use LangChain, an orchestrator for language pipelines, to build an assistant capable of loading information from a web page and use it for answering user questions

# Step 0: Configuring the environment
This step install the necessary libraries for connecting with Galileo and the models. By using our Local GenAI workspace image, most of the libraries are already pre installing - in our case, we just need to add the connector to work with PDF documents

In [20]:
!pip install PyPDF
!pip install galileo-observe
!pip install galileo-protect

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


### Configuration of Hugging face caches

In the next cell, we configure HuggingFace cache, so that all the models downloaded from them are persisted locally, even after the workspace is closed. This is a future desired feature for AI Studio and the GenAI addon.

In [1]:
import os
os.environ["HF_HOME"] = "/home/jovyan/local/hugging_face"
os.environ["HF_HUB_CACHE"] = "/home/jovyan/local/hugging_face/hub"

## Step 1: Data Loading

In this step, we will use the Langchain framework to  extract the content from a local PDF file with the product documentation. Also, we have commented some example on how to use Web Loaders to load data form pages on the web.

In [2]:
from langchain.document_loaders import WebBaseLoader
from langchain_community.document_loaders import PyPDFLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
file_path = (
    "./AIStudioDoc.pdf"
)
pdf_loader = PyPDFLoader(file_path)
pdf_data = pdf_loader.load()

#loader1 = WebBaseLoader("https://www.hp.com/us-en/workstations/ai-studio.html") # If you want to change the knowledge base, just modify this link.
#data1 = loader1.load()

#loader2 = WebBaseLoader("https://zdocs.datascience.hp.com/docs/aistudio")
#data2 = loader2.load()

## Step 2: Creation of Chunks
Here, we split the loaded documents into chunks, so we have smaller and more specific texts to add do our vector database.

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [5]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
splits = text_splitter.split_documents(pdf_data)



## Step 3: Retrieval

We transform the texts into embeddings and store them in a vector database. This allows us to perform similarity search, and proper retrieval of documents


In [6]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma


In [7]:
embedding = HuggingFaceEmbeddings()

/tmp/ipykernel_70730/548774098.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings()
/tmp/ipykernel_70730/548774098.py:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embedding = HuggingFaceEmbeddings()
/opt/conda/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)

In [8]:
vectordb = Chroma.from_documents(documents=splits, embedding=embedding)
retriever = vectordb.as_retriever()


## Step 4: Model

In this example, we will use OpenAI API to connect to GPT-3.5 model. A broader range of models could be used.

In [33]:
import os
from langchain_openai import OpenAI

os.environ["OPENAI_API_KEY"] = "" #your api key
llm = OpenAI(model_name="gpt-3.5-turbo-instruct")


In [25]:
### Alternate code to connect to Hugging Face models

#import yaml
#with open('config.yaml') as file:
    #config = yaml.safe_load(file)
#huggingfacehub_api_token = config["hf_key"]
#repo_id = "mistralai/Mistral-7B-Instruct-v0.2"
#llm = HuggingFaceEndpoint(
   #huggingfacehub_api_token=huggingfacehub_api_token,
   #repo_id=repo_id,
#)


In [12]:
###Alternate code to load local models

#from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler
#from langchain_community.llms import LlamaCpp

#callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

#llm = LlamaCpp(
#            model_path="/home/jovyan/datafabric/llama2-7b/ggml-model-f16-Q5_K_M.gguf",
#            n_gpu_layers=30,
#            n_batch=512,
#            n_ctx=4096,
#            max_tokens=1024,
#            f16_kv=True,  
#            callback_manager=callback_manager,
#            verbose=False,
#            stop=[],
#            streaming=False,
#            temperature=0.2,
#        )    

## Step 5: Chain
In this part, we define a pipeline that receives a question and context, formats the context documents, and uses a Hugging Face (Mistral) chat model to answer the question based on the provided context. The output is then formatted as a string for easy reading.

In [40]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from typing import List
from langchain.schema.document import Document

def format_docs(docs: List[Document]) -> str:
    return "\n\n".join([d.page_content for d in docs])

template = """You are an virtual Assistant for a Data Science platform called AI Studio. Answer the question based on the following context:

    {context}

    Question: {query}
    """
prompt = ChatPromptTemplate.from_template(template)

chain = {"context": retriever | format_docs, "query": RunnablePassthrough()} | prompt | llm | StrOutputParser()

## Step 6: Connect to Galileo
Through the Galileo library called Prompt Quality, we connect our API generated in the Galileo console to log in. To get your ApiKey, use this link: https://console.hp.galileocloud.io/api-keys

In [17]:
import promptquality as pq

os.environ['GALILEO_API_KEY'] = "" #your api Key
galileo_url = "" #your console link
pq.login(galileo_url)

👋 You have logged into 🔭 Galileo (https://console.hp.galileocloud.io/) as diogo.vieira@hp.com.


Config(console_url=Url('https://console.hp.galileocloud.io/'), username=None, password=None, api_key=SecretStr('**********'), token=SecretStr('**********'), current_user='diogo.vieira@hp.com', current_project_id=None, current_project_name=None, current_run_id=None, current_run_name=None, current_run_url=None, current_run_task_type=None, current_template_id=None, current_template_name=None, current_template_version_id=None, current_template_version=None, current_template=None, current_dataset_id=None, current_job_id=None, current_prompt_optimization_job_id=None, api_url=Url('https://api.hp.galileocloud.io/'))

Through callbacks, we choose the metrics we want to monitor via the Galileo console. We pass a list of queries to run our created chain and log in to Galileo.


### Galileo Evaluate

Galileo Evaluate is a platform designed to optimize and simplify the experimentation and evaluation of generative AI systems, especially large language model (LLM) applications. Its goal is to facilitate the process of building AI systems with deep insights and collaborative tools, replacing fragmented experimentation in spreadsheets and notebooks with a more integrated approach.

You can log metrics in Galileo Evaluate and track all your experiments in one place.

In our example, we logged several questions, selected specific metrics, and ran a batch of experiments to evaluate our chain.

To learn more about the available metrics, see: [Galileo Guardrail Metrics](https://docs.rungalileo.io/galileo/gen-ai-studio-products/galileo-guardrail-metrics).



In [18]:
# Create callback handler
prompt_handler = pq.GalileoPromptCallback(
    project_name="AIStudio_Chatbot_template",
    scorers=[pq.Scorers.context_adherence_luna, pq.Scorers.correctness, pq.Scorers.toxicity, pq.Scorers.sexist]
)

# Run your chain experiments across multiple inputs with the galileo callback
inputs = [
    "What is AI Studio",
    "How to create projects in AI Studio?"
    "How to monitor experiments?",
    "What are the different workspaces available?",
    "What, exactly, is a workspace?",
    "How to share my experiments with my team?",
    "Can I access my Git repository?",
    "Do I have access to files on my local computer?",
    "How do I access files on the cloud?",
    "Can I invite more people to my team?"
]
chain.batch(inputs, config=dict(callbacks=[prompt_handler]))

# publish the results of your run
prompt_handler.finish()

Processing chain run...:   0%|          | 0/5 [00:00<?, ?it/s]

Initial job complete, executing scorers asynchronously. Current status:
rag_nli: Done ✅
cost: Done ✅
toxicity: Done ✅
sexist: Done ✅
pii: Done ✅
protect_status: Done ✅
latency: Done ✅
factuality: Failed ❌, error was: Executing this metric requires credentials for OpenAI, Azure OpenAI or Vertex to be set.
🔭 View your prompt run on the Galileo console at: https://console.hp.galileocloud.io/prompt/chains/9c4b6241-f741-4f82-a31c-e5cd31cadf70/034bb37d-9a3a-4bf7-a0a3-83402b7b8ec5?taskType=12


### Galileo Protect

Galileo Protect serves as a powerful tool for safeguarding AI model outputs by detecting and preventing the release of sensitive information like personal addresses or other PII. By integrating Galileo Protect into your AI pipelines, you can ensure that model responses comply with privacy and security guidelines in real-time.

Galileo functions as an API that provides support for protection verification of your chain/LLM. To log into the Galileo console, it is necessary to integrate it with another service, such as Galileo Evaluate or Galileo Observe.

**Attention**: an integrated API within the Galileo console is required to perform this verification.

In [28]:
import galileo_protect as gp
import os

project = gp.create_project('ais')
project_id = project.id

stage = gp.create_stage(name="ais", project_id=project_id)
stage_id = stage.id


Galileo Protect works by creating rules that identify conditions such as Personally Identifiable Information (PII) and toxicity. It ensures that the prompt will not receive or respond to sensitive questions. In this example, we create a set of rules (ruleset) and a set of actions that return a pre-programmed response if a rule is triggered. Galileo Protect also offers a variety of other metrics to suit different protection needs. You can learn more about the available metrics here: [Supported Metrics and Operators](https://docs.rungalileo.io/galileo/gen-ai-studio-products/galileo-protect/how-to/supported-metrics-and-operators).

Additionally, it is possible to import rulesets directly from Galileo through stages. Learn more about this feature here: [Invoking Rulesets](https://docs.rungalileo.io/galileo/gen-ai-studio-products/galileo-protect/how-to/invoking-rulesets).


In [31]:
import galileo_protect as gp
from galileo_protect import OverrideAction, ProtectTool, ProtectParser, Ruleset

stage_id = stage.id  
project_id = project.id 

protect_tool = ProtectTool(
    stage_id=stage_id,  
    prioritized_rulesets=[
        Ruleset(rules=[
            {
                "metric": "pii",
                "operator": "contains",
                "target_value": "ssn",
            },
        ],
        action={
            "type": "OVERRIDE",
            "choices": [
                "Personal Identifiable Information detected in the model output. Sorry, I cannot answer that question."
            ],
        }),
    ],
    timeout=10
)

protect_parser = ProtectParser(chain=chain)

protected_chain = protect_tool | protect_parser.parser

protected_chain.invoke({"input": "What's my SSN? Hint: my SSN is 123-45-6789", "output": "Your SSN is 123-45-6789"})


'Personal Identifiable Information detected in the model output. Sorry, I cannot answer that question.'

### Galileo Observe

Galileo Observe helps you monitor your generative AI applications in production. With Observe you will understand how your users are using your application and identify where things are going wrong. Keep tabs on your production system, instantly receive alerts when bad things happen, and perform deep root cause analysis though the Observe dashboard.

You can connect Galileo Observe to your Langchain chain to monitor metrics such as cost and guardrail indicators.

In [42]:
#Connecting Langchain with Galileo Observe
from galileo_observe import GalileoObserveCallback

monitor_handler = GalileoObserveCallback(project_name="test_galileo_observe")

example_query = "What is AI Studio?"

output = chain.invoke(
    example_query,
    config=dict(callbacks=[monitor_handler])
)

output


'\nAnswer: AI Studio is a standalone application specifically developed for data scientists and engineers to connect to multiple data-stores across local and cloud networks and perform computations locally without interruption. It is part of the Z by HP AI Studio platform.'

### Galileo Protect + Evaluate

Here, we combined Galileo Protect with Galileo Evaluate and ran a batch of sensitive questions to see Galileo Protect in action.


In [48]:
prompt_handler = pq.GalileoPromptCallback(
    project_name="test_galileo_evaluate",
    scorers=[pq.Scorers.context_adherence_luna, pq.Scorers.correctness, pq.Scorers.toxicity, pq.Scorers.sexist, pq.Scorers.pii]
)

# List of inputs with different variants of SSN queries
inputs = [
    {"input": "What's my SSN? Hint: my SSN is 123-45-6789", "output": "Your SSN is 123-45-6789"},
    {"input": "Can you confirm my SSN is 987-65-4321?", "output": "Your SSN is 987-65-4321"},
    {"input": "I'm trying to remember my SSN, I think it's 111-22-3333", "output": "Your SSN is 111-22-3333"},
    {"input": "Is 222-33-4444 my SSN?", "output": "Yes, that is your SSN."},
    {"input": "Could you look up my SSN: 555-66-7777?", "output": "Your SSN is 555-66-7777"},
]

# Running the batch chain with GalileoPromptCallback
protected_chain.batch(inputs, config=dict(callbacks=[prompt_handler]))

# Finalizing and publishing the results
prompt_handler.finish()


Processing chain run...:   0%|          | 0/5 [00:00<?, ?it/s]

Initial job complete, executing scorers asynchronously. Current status:
rag_nli: Done ✅
cost: Done ✅
toxicity: Done ✅
sexist: Done ✅
pii: Done ✅
protect_status: Done ✅
latency: Done ✅
factuality: Computing 🚧
🔭 View your prompt run on the Galileo console at: https://console.hp.galileocloud.io/prompt/chains/f81f0f16-52ee-48a8-ad28-d2752fbcf771/a0a9851c-4787-4a6d-b543-585bea5b71a0?taskType=12


## Model service Galileo Protect + Observe

In [66]:
import mlflow
from mlflow.types.schema import Schema, ColSpec
from mlflow.models import ModelSignature
import promptquality as pq
import galileo_protect as gp
from galileo_protect import ProtectTool, ProtectParser, Ruleset
from galileo_observe import GalileoObserveCallback
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain_community.llms import LlamaCpp
from langchain.vectorstores import Chroma
from typing import List
import os

class AIStudioModelService(mlflow.pyfunc.PythonModel):

    def load_context(self, context):
        os.environ['GALILEO_API_KEY'] = "" #your api key
        os.environ['GALILEO_CONSOLE_URL'] = "" #your console api key

        # Load the LlamaCpp model locally
        self.model_path = context.artifacts["models"]
        self.llm_model = LlamaCpp(
            model_path=self.model_path,
            n_gpu_layers=30,
            n_batch=512,
            n_ctx=4096,
            max_tokens=1024,
            f16_kv=True,
            temperature=0.2
        )

        # Set up the vector store retriever
        self.vector_store = Chroma(persist_directory="./chroma_db")
        self.retriever = self.vector_store.as_retriever()

        # Set up Galileo Observe Callback for monitoring
        self.monitor_handler = GalileoObserveCallback(project_name="AIStudio_Chatbot_template")

        # Set up Galileo Protect for protecting the model responses
        project = gp.create_project('ais')
        stage = gp.create_stage(name="ais", project_id=project.id)
        self.protect_tool = ProtectTool(
            stage_id=stage.id,
            prioritized_rulesets=[
                Ruleset(rules=[
                    {
                        "metric": "pii",
                        "operator": "contains",
                        "target_value": "ssn",
                    },
                ],
                action={
                    "type": "OVERRIDE",
                    "choices": [
                        "Personal Identifiable Information detected in the model output. Sorry, I cannot answer that question."
                    ],
                }),
            ],
            timeout=10
        )

    def predict(self, context, model_input):
        # Retrieve relevant documents for context
        retrieved_docs = self.retriever.get_relevant_documents(model_input["question"])
        context_docs = "\n\n".join([doc.page_content for doc in retrieved_docs])

        # Define prompt template
        template = """You are a virtual Assistant for a Data Science platform called AI Studio. Answer the question based on the following context:

        {context}

        Question: {query}
        """
        prompt = ChatPromptTemplate.from_template(template)

        chain = {
            "context": lambda inputs: context_docs,
            "query": RunnablePassthrough()
        } | prompt | self.llm_model | StrOutputParser()

        # Protecting the chain
        protect_parser = ProtectParser(chain=chain)
        protected_chain = self.protect_tool | protect_parser.parser

        # Invoke the protected chain
        result = protected_chain.invoke(
            {"input": model_input["question"], "output": ""},
            config=dict(callbacks=[self.monitor_handler])
        )

        # Finalize the observation callback
        self.monitor_handler.finish()

        return {"result": result}

    @classmethod
    def log_model(cls, model_folder):
        input_schema = Schema([ColSpec("string", "question")])
        output_schema = Schema([ColSpec("string", "result")])
        signature = ModelSignature(inputs=input_schema, outputs=output_schema)

        artifacts = {"models": model_folder}
        mlflow.pyfunc.log_model(
            artifact_path="AIStudio_with_LlamaCpp",
            python_model=cls(),
            artifacts=artifacts,
            signature=signature,
            pip_requirements=[
                "mlflow==2.9.2",
                "langchain==0.3.6",
                "promptquality==0.65.0",
                "galileo-protect==0.14.2",
                "galileo-observe==1.3.0",
                "chromadb"
            ],
        )

# Logging and registering the model with MLflow
mlflow.set_experiment(experiment_name='AIStudio_with_LlamaCpp')

artifact_path = "AIStudio_with_LlamaCpp"
with mlflow.start_run(run_name='AIStudio_Model_with_LlamaCpp') as run:
    AIStudioModelService.log_model(
        model_folder='/path/to/llama-model-directory'
    )

    mlflow.register_model(
        model_uri=f"runs:/{run.info.run_id}/AIStudio_with_LlamaCpp",
        name="AIStudio_Model_with_LlamaCpp"
    )


<Experiment: artifact_location='/phoenix/mlflow/675251160511990687', creation_time=1730360944027, experiment_id='675251160511990687', last_update_time=1730360944027, lifecycle_stage='active', name='AIStudio_with_Protect_Observe', tags={}>